In [ ]:
!pip install mysqlclient

     |████████████████████████████████| 87 kB 3.6 MB/s 
  Created wheel for mysqlclient: filename=mysqlclient-2.1.0-cp37-cp37m-linux_x86_64.whl size=99965 sha256=c8b4f918fd525058d0a9ef2bbf9b6e62a89a2c314ed4256353e48ac83b0edae8
  Stored in directory: /root/.cache/pip/wheels/97/d4/df/08cd6e1fa4a8691b268ab254bd0fa589827ab5b65638c010b4
Successfully built mysqlclient


In [ ]:
import pandas as pd
import numpy as np
import sqlalchemy as sql_a

In [ ]:
# Connect database

username = ""
password = ""
host = "" 
port = 0
database_name = ""


con_str = f"mysql://{username}:{password}@{host}:{port}/{database_name}"
con = sql_a.create_engine(con_str, connect_args={'connect_timeout': 5})

In [ ]:
reviews = pd.read_sql("SELECT hotel_id, user_id, labels FROM coba_review", con=con)
reviews

,hotel_id,user_id,labels
0,21,6,1
1,87,95,0
2,27,65,1
3,53,70,1
4,9,12,1
...,...,...,...
9851,14,82,1
9852,10,28,0
9853,51,40,0
9854,22,94,0


In [ ]:
df_user_id = pd.read_sql("SELECT id FROM Users", con=con)
df_user_id

,id
0,1
1,2
2,3
3,4
4,5
...,...
108,110
109,114
110,117
111,119


In [ ]:
df_hotel_id = pd.read_sql("SELECT id FROM hotel_dummy_photos", con=con)
df_hotel_id

,id
0,1
1,2
2,3
3,4
4,5
...,...
95,96
96,97
97,98
98,99


In [ ]:
def create_matrix(df_in: pd.DataFrame, users_id: pd.DataFrame, hotels_id: pd.DataFrame):
  """
  Will create sentiment matrix of the users and the hotel. 
  It will returned matrix filled with 0, 0.5, 1 (0 negative, 0.5 not reviewed / neutral, 1 positives)
  """
  df = df_in
  user_id = users_id["id"].to_list()
  hotels_id = hotels_id["id"].to_list()

  data = {
      "user_id_cols" : user_id
  }
  matrix = pd.DataFrame(data).set_index("user_id_cols", drop=True)
  num_users = len(user_id)

  for i in hotels_id:
    # Init column
    temp_arr = np.empty(num_users)
    temp_arr[:] = 0.5
    matrix[i] = temp_arr

    # Fill User and sentiment
    user_sentiments = df[df["hotel_id"] == i][["user_id", "labels"]].values
    for j in user_sentiments:
      # Example of user_sentiments
      # ([[ 67,   1],
      #  [ 96,   0],
      #  [ 83,   1],
      #  [  6,   1],
      #  [ 31,   1], ...
      matrix.at[j[0], i] = j[1]

  return matrix.copy()

matrix = create_matrix(reviews, df_user_id, df_hotel_id)
matrix

,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
user_id_cols,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.5,0.0,1.0,0.0,0.0,1.0,1.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
2,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.5,1.0,...,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,...,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,...,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0
5,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,...,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,...,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5
114,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,...,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5
117,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,...,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5


In [ ]:
# Pivoted user
user = 107

In [ ]:
# Similarities between pivoted user and other users
from sklearn.metrics.pairwise import cosine_similarity

def get_user_similarity(matrix: pd.DataFrame, user_id: int):
  y = matrix.loc[user_id].values
  x = matrix.values
  return cosine_similarity(x, y.reshape(1,-1)).reshape(1,-1)[0]

similarity = get_user_similarity(matrix, user)
similarity

array([0.75670397, 0.7466668 , 0.7193832 , 0.74161985, 0.79215531,
       0.71066905, 0.72111026, 0.74501401, 0.71066905, 0.69822128,
       0.76488135, 0.77942952, 0.7842457 , 0.69822128, 0.7466668 ,
       0.73146949, 0.76488135, 0.76328181, 0.73994021, 0.73146949,
       0.74161985, 0.76811457, 0.73146949, 0.76328181, 0.69822128,
       0.75333333, 0.79059512, 0.79844216, 0.7466668 , 0.73484692,
       0.79535595, 0.73994021, 0.8       , 0.74501401, 0.68924144,
       0.70895373, 0.76980348, 0.74161985, 0.74174705, 0.76157731,
       0.75333333, 0.76157731, 0.69282032, 0.73994021, 0.73315192,
       0.70534579, 0.6964572 , 0.71066905, 0.76811457, 0.78581817,
       0.75169478, 0.7176695 , 0.6964572 , 0.7484552 , 0.77784447,
       0.68194937, 0.8       , 0.78268394, 0.76649249, 0.79059512,
       0.79372539, 0.6964572 , 0.79689397, 0.69822128, 0.78102497,
       0.74833148, 0.7466668 , 0.72111026, 0.72801099, 0.79689397,
       0.77784447, 0.81853528, 0.74161985, 0.80161689, 0.76328

In [ ]:
# Let's try to fill the not reviewd hotel
# First we need to see what index are the hotel is not reviewd

def get_hotel_not_reviewed(matrix: pd.DataFrame, user_id: int):
  return matrix.loc[user_id][matrix.loc[user_id] == 0.5].index.values  

not_reviewed = get_hotel_not_reviewed(matrix, user)
not_reviewed

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100])

In [ ]:
from typing import List
def give_recommendation(num_recs:int, matrix: np.ndarray, 
                        hotel_not_reviewed: np.ndarray, 
                        user_weights: np.ndarray) -> List:
  # Let's try to calculate predicted sentiment (higher is better becasue 1 is positive and 0 is negative)
  predicted_values = np.zeros(len(hotel_not_reviewed))
  for i in range(len(hotel_not_reviewed)):
    predicted_values[i] = (matrix[matrix.columns[i]].values @ similarity) / sum(similarity)
  hotel_order = np.argsort(predicted_values)[::-1][:len(hotel_not_reviewed)]
  final_recs = []
  for i in range(min([len(hotel_not_reviewed), num_recs])):
    final_recs.append(hotel_not_reviewed[hotel_order[i]])

  return final_recs

give_recommendation(10, matrix, not_reviewed, similarity)

[53]

In [ ]:
# Pipelines
%%prun
user_id = 107
matrix = create_matrix(reviews, df_user_id, df_hotel_id)
similarity = get_user_similarity(matrix, user_id)
not_reviewed = get_hotel_not_reviewed(matrix, user_id)
give_recommendation(10, matrix, not_reviewed, similarity)